In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.3.0
      /_/

Using Python version 2.7.6 (default, Mar 22 2014 22:59:56)
SparkContext available as sc, HiveContext available as sqlCtx.


In [2]:
# исходные данные 
user_id = 387

In [3]:
# файл с оценками
# user id | item id | rating | timestamp
user_marks_src = sc.textFile("/recsys/ml-100k/u.data")
user_marks_src.first()

u'196\t242\t3\t881250949'

In [4]:
# файл с фильмами
film_data_src = sc.textFile("/recsys/ml-100k/u.item")
film_data_src.first()

u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

In [5]:
# id, name
film_data = film_data_src.map(lambda x: (int(x.split('|')[0]), x.split('|')[1]))
film_data.take(5)

[(1, u'Toy Story (1995)'),
 (2, u'GoldenEye (1995)'),
 (3, u'Four Rooms (1995)'),
 (4, u'Get Shorty (1995)'),
 (5, u'Copycat (1995)')]

In [6]:
# user id | item id | rating | timestamp
user_marks = user_marks_src.map(lambda x: [int(i) for i in x.split('\t')])
user_marks.first()

[196, 242, 3, 881250949]

##Часть1. Найдем основные характеристики данных movielens

####Количество всех пользователей и количество всех фильмов в данных

In [7]:
# общее кол-во пользователей
user_count = user_marks.map(lambda x: x[0]).distinct().count()
print user_count

943


In [8]:
# кол-во фильмов
film_count = film_data.count()
print film_count

1682


####Сколько пользователь в среднем ставит рейтингов? (количество рейтингов/количество пользователей) – average_user_ratings

In [9]:
average_user_ratings = round(float(user_marks.count()) / user_count, 4)
print average_user_ratings

106.0445


####Сколько фильм в среднем имеет рейтингов (количество рейтингов/количество фильмов)? – average_film_ratings

In [10]:
average_film_ratings = round(float(user_marks.count()) / film_count, 4)
print average_film_ratings

59.453


####Процент заполненных ячеек в данных (количество рейтингов/(количество пользователей*количество фильмов)) – completeness

In [11]:
completeness = round(float(user_marks.count()) / (user_count*film_count), 4)
print completeness

0.063


##Часть 2. User-user CF

####Средний рейтинг для каждого пользователя

In [12]:
from operator import add, itemgetter
user_ratings_counts = user_marks.map(lambda x: (x[0], 1)).reduceByKey(add)
user_ratings_counts.take(3)

[(2, 62), (4, 24), (6, 211)]

In [13]:
user_ratings_sum = user_marks.map(lambda x: (x[0], x[2])).reduceByKey(add)
user_ratings_sum.take(3)

[(2, 230), (4, 104), (6, 767)]

In [14]:
user_avg = user_ratings_sum.join(user_ratings_counts).map(lambda x: (x[0], round(float(x[1][0]) / x[1][1], 4)))
user_avg.take(3)

[(2, 3.7097), (4, 4.3333), (6, 3.6351)]

####Корреляция Пирсона

In [15]:
# средняя оценка моего юзера
my_user_avg = user_avg.filter(lambda x: x[0] == user_id).collect()[0][1]
print my_user_avg

3.3618


In [16]:
# film_id, rating
my_user_marks = user_marks.filter(lambda x: x[0] == user_id).map(lambda x: (x[1], x[2]))
my_user_marks.count()

304

In [17]:
# общие фильмы и оценки
# film_id, (rating, my_rating)
def get_common_films(_uid):
    return user_marks.filter(lambda x: x[0] == _uid).map(lambda x: (x[1], x[2])).join(my_user_marks)

In [18]:
# средние оценки других пользователей
other_user_avg = user_avg.filter(lambda x: x[0] is not user_id)
other_user_avg.take(3)

[(2, 3.7097), (4, 4.3333), (6, 3.6351)]

In [19]:
other_user_avg_array = other_user_avg.collect()
up_list = []
print len(other_user_avg_array)

943


In [20]:
# сделаем broadcast-словарь со всеми средними оценками
all_neighbour_avg = user_avg.collect()
all_avg_dict = {}
for _i in all_neighbour_avg:
    all_avg_dict[_i[0]] = _i[1]

all_avg_dict_br = sc.broadcast(all_avg_dict)
print len(all_avg_dict_br.value)

943


In [21]:
from math import sqrt
import cPickle
from pyspark.mllib.stat import Statistics

def per(film_item):
    global sum1
    global sum2
    global sum3
    user_film_rating = film_item[1][0]
    my_film_rating = film_item[1][1]
    sum1 += (my_film_rating - my_user_avg_br.value) * (user_film_rating - user_avg_rating_br.value)
    sum2 += (my_film_rating - my_user_avg_br.value) ** 2
    sum3 += (user_film_rating - user_avg_rating_br.value) ** 2

result = []

with open('pearson_2.picle', "r") as _dump:
    result = cPickle.load(_dump)
    
if len(result) == 0:    
    
    # идем по всем средним оценкам пользователей
    __c = 0
    for _item in other_user_avg_array:

        __c += 1

        # пользователь
        uid = _item[0]

        # Общие фильмы с моим пользователем
        common_films = get_common_films(uid)

        # средние оценки по общим фильмам
        common_films_count = common_films.count()
    #     my_avg = common_films.map(lambda x: (1, x[1][1])).reduceByKey(add).collect()[0][1]
    #     my_avg = round(float(my_avg) / common_films_count, 4)
        my_avg = my_user_avg

    #     u_avg = common_films.map(lambda x: (1, x[1][0])).reduceByKey(add).collect()[0][1]
    #     u_avg = round(float(u_avg) / common_films_count, 4)
        u_avg = all_avg_dict.get(uid)

        # руками 1
        sum1 = sc.accumulator(0)
        sum2 = sc.accumulator(0)
        sum3 = sc.accumulator(0)

        my_user_avg_br = sc.broadcast(my_avg)
        user_avg_rating_br = sc.broadcast(u_avg)

        common_films.foreach(per)
        if sum2.value == 0 or sum3.value == 0:
            pears = 0
        else:        
            pears = round(float(sum1.value) / (sqrt(sum2.value) * sqrt(sum3.value)), 4)

        _k = min(round(float(common_films_count)/50, 4), 1)
        new_pears = round(_k * pears, 4)

        res = (uid, pears, new_pears)
        result.append(res)

        if __c % 100 == 0:
            print __c * 100 / len(other_user_avg_array)
    
        
        with open('pearson_2.picle', "w") as _dump:
            cPickle.dump(result, _dump, cPickle.HIGHEST_PROTOCOL)

In [22]:
sorted_result = sorted(result, key=itemgetter(2), reverse=True)
sorted_result_30 =  sorted_result[1:31]
pearson_neighbours = map(lambda x: x[0], sorted_result_30)
print pearson_neighbours

[267, 397, 776, 177, 367, 118, 854, 486, 77, 442, 41, 457, 339, 833, 323, 622, 21, 23, 94, 158, 766, 49, 875, 566, 297, 933, 425, 7, 781, 868]


#### Прогноз оценок

In [23]:
# фильмы, которые оценил
my_user_films_list = my_user_marks.map(lambda x: x[0]).collect()

# фильмы которые не оценил
no_rating_films = film_data.filter(lambda x: x[0] not in my_user_films_list)
no_rating_films_list = no_rating_films.map(lambda x: x[0]).collect()
print len(no_rating_films_list)

1378


In [24]:
# найдем оценки соседей для моих неоцененных фильмов
no_rating_films_neighbour_rating = user_marks\
    .filter(lambda x: x[1] in no_rating_films_list)\
    .map(lambda x: (x[1], (x[0], x[2])))

no_rating_films_neighbour_rating.take(3)

[(242, (196, 3)), (302, (186, 3)), (377, (22, 1))]

In [25]:
# сделаем broadcast-словарь с оценками пользователей фильмов, которые нам интересны
neighbour_rating = no_rating_films_neighbour_rating\
    .groupByKey()\
    .map(lambda x : (x[0], list(x[1])))\
    .collect()
    
rating_dict = {}
for films_r in neighbour_rating:
    f_id = films_r[0]
    rating_dict[f_id] = {}
    for user_r in films_r[1]:
        u_id = user_r[0]
        rating_dict[f_id][u_id] = user_r[1]
        

rating_dict_br = sc.broadcast(rating_dict)        
print len(rating_dict_br.value)

1378


In [26]:
# сделаем broadcast-словарь со средними оценками
neighbour_avg = user_avg.filter(lambda x: x[0] in pearson_neighbours).collect()
avg_dict = {}
for _i in neighbour_avg:
    avg_dict[_i[0]] = _i[1]

avg_dict_br = sc.broadcast(avg_dict)
print avg_dict_br.value

{7: 3.9653, 776: 3.4667, 267: 3.9351, 781: 3.7857, 21: 2.6704, 23: 3.6358, 158: 3.8035, 933: 2.6467, 49: 2.6837, 425: 2.9559, 177: 3.6174, 566: 3.443, 442: 3.1259, 833: 3.0562, 323: 3.618, 457: 4.0253, 77: 3.6111, 397: 3.7059, 339: 4.0039, 854: 3.212, 297: 3.4375, 94: 3.6575, 868: 2.9519, 486: 3.2222, 875: 4.191, 622: 3.3904, 367: 4.1552, 118: 4.662, 41: 3.7692, 766: 3.3543}


In [27]:
def prabably_rating(film_id):
    s1 = 0
    s2 = 0
    # идем по всем соседям
    for neighbour in sorted_result_30:
        uid = neighbour[0]
        p = neighbour[2]
        # надо найти оценку пользователя, его среднюю оценку
        user_rating = rating_dict_br.value.get(film_id, {}).get(uid, None)
        
        if user_rating is None:
            continue
        
        user_avg_rating = avg_dict_br.value.get(uid)
        
        s1 += p * (user_rating - user_avg_rating)
        s2 += abs(p)
    if s2 == 0:
        return (film_id, my_user_avg)
    return (film_id, round(my_user_avg + (float(s1) / s2), 4))

In [28]:
no_rating_films.take(4)

[(3, u'Four Rooms (1995)'),
 (5, u'Copycat (1995)'),
 (6, u'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'),
 (9, u'Dead Man Walking (1995)')]

In [29]:
res = no_rating_films.map(lambda x: x[0]).map(prabably_rating).sortBy(lambda x: -x[1])
res.take(12)

[(922, 5.2662),
 (671, 5.1448),
 (251, 5.1396),
 (880, 5.1396),
 (887, 5.1396),
 (1137, 5.1396),
 (1142, 5.1396),
 (1405, 5.1396),
 (1598, 5.1396),
 (165, 4.9457),
 (1193, 4.9188),
 (646, 4.765)]

In [30]:
pearson_top10 = res.map(lambda x: x[0]). take(10)
pearson_top10

[922, 671, 251, 880, 887, 1137, 1142, 1405, 1598, 165]

##Часть 3. Базовые предикторы

####Глобальное среднее (average_rating) по всему датасету

In [31]:
average_rating_s = float(user_marks.map(lambda x: x[2]).sum()) / user_marks.map(lambda x: 1).sum()
average_rating = round(average_rating_s, 4)
print average_rating

3.5299


####Базовый предиктор для всех пользователей

In [32]:
average_rating_br = sc.broadcast(average_rating)
# пользователь (сумма, кол-во оценок)
pre_marks = user_marks\
    .map(lambda x: (x[0], (x[2] - average_rating_br.value, 1)))\
    .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))

pre_marks.take(4)

[(2, (11.146200000000006, 62)),
 (4, (19.282400000000003, 24)),
 (6, (22.191099999999928, 211)),
 (8, (15.735899999999987, 59))]

In [33]:
bu_rdd = pre_marks.map(lambda x: (x[0], round(1.0/(10 + x[1][1]) * x[1][0], 4)))
print bu_rdd.take(3), bu_rdd.count()

[(2, 0.1548), (4, 0.5671), (6, 0.1004)] 943


####Базовый предиктор для всех фильмов

In [34]:
bu = bu_rdd.collect()
bu_dict = {}
for _b in bu:
    bu_dict[_b[0]] = _b[1]

bu_dict_br = sc.broadcast(bu_dict)

In [35]:
# фильм (сумма, кол-во пользователей)
pre_marks_2 = user_marks\
    .map(lambda x: (x[1], (x[2] - average_rating_br.value - bu_dict_br.value.get(x[0]), 1)))\
    .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))

pre_marks_2.take(3)

[(2, (-33.23320000000001, 131)),
 (4, (-4.659500000000008, 209)),
 (6, (2.9346999999999994, 26))]

In [36]:
bi_rdd = pre_marks_2.map(lambda x: (x[0], round(1.0/(25 + x[1][1]) * x[1][0], 4)))
print bi_rdd.take(3), bi_rdd.count()

[(2, -0.213), (4, -0.0199), (6, 0.0575)] 1682


In [37]:
bi = bi_rdd.collect()
bi_dict = {}
for _b in bi:
    bi_dict[_b[0]] = _b[1]

bi_dict_br = sc.broadcast(bi_dict)

##Часть 4. Item-item CF

#### Cделаем табличку с рейтингами

In [38]:
# из пользовательских оценок вычтем предикторы
new_user_marks = user_marks\
    .map(
        lambda x: (x[0], x[1], 
                   round(x[2] - average_rating_br.value - bi_dict_br.value.get(x[1]) - bu_dict_br.value.get(x[0]), 4)
                  )
    )
new_user_marks.take(3)

[(196, 242, -0.9886), (186, 302, -1.0289), (22, 377, -1.9717)]

In [39]:
new_user_marks_list = new_user_marks.collect()

In [40]:
from collections import defaultdict
rating_dict = defaultdict(dict)
for _i in new_user_marks_list:
    _uid = _i[0]
    _fid = _i[1]
    _r = _i[2]
    rating_dict[_fid][_uid] = _r

rating_dict[242][196]

-0.9886

In [45]:
from pyspark.sql import Row

# список пользователей
user_list = user_marks.map(lambda x: x[0]).distinct().collect()

# список фильмов
film_list = film_data.map(lambda x: x[0]).collect()

user_list_br = sc.broadcast(user_list)

rd_br = sc.broadcast(rating_dict)

# косинус для двух фильмов
def cos(film_1, film_2):
    s1 = 0
    s2 = 0
    s3 = 0
    for u in user_list_br.value:
        r_u1 = rd_br.value.get(film_1,{}).get(u, 0)
        r_u2 = rd_br.value.get(film_2,{}).get(u, 0)
#         b_u1 = average_rating_br.value + bu_dict_br.value.get(u) + bi_dict_br.value.get(film_1)
#         b_u2 = average_rating_br.value + bu_dict_br.value.get(u) + bi_dict_br.value.get(film_2)
#         s1 += (r_u1 - b_u1) * (r_u2 - b_u2)
#         s2 += (r_u1 - b_u1) ** 2
#         s3 += (r_u2 - b_u2) ** 2
        s1 += r_u1 * r_u2
        s2 += (r_u1) ** 2
        s3 += (r_u2) ** 2
    
    if s2 == 0 or s3 == 0:
        return 0
    
    return round(float(s1) / (sqrt(s2) * sqrt(s3)), 4)

def cos_f(film):
    _r = {}
    for f in film_list:
        _r[f] = cos(film, f)
    return _r

def cos_f_2(film):
    _r = {}
    _rdd = film_data.map(lambda x: {x[0]: cos(film, x[0])})
    _local = _rdd.collect()
    return _local

In [47]:
cos_dict = {}

with open('cos_dict.picle', "r") as _dump:
    cos_dict = cPickle.load(_dump)


if len(cos_dict) == 0:
    r = {}
    _counter = 0
    for fid in film_list:
        r[fid] = cos_f_2(fid)
        _counter += 1
        if _counter % 50 == 0:
            print _counter * 100 / len(film_list)

    # немного модифицируем
    for fid, values in r.iteritems():
        cos_dict[fid] = {}
        for _cos in values:
            f = _cos.keys()[0]
            v = _cos.values()[0]
            cos_dict[fid][f] = v

    with open('cos_dict.picle', "w") as _dump:
        cPickle.dump(cos_dict, _dump, cPickle.HIGHEST_PROTOCOL)

2
5
8
11
14
17
20
23
26
29
32
35
38
41
44
47
50
53
56
59
62
65
68
71
74
77
80
83
86
89
92
95
98


In [65]:
sqlCtx

####Для каждого фильма, по которому у данного пользователя не стоит рейтинг

####30 ближайших фильмов-соседей для этого фильма (среди всех фильмов, а не фильмов, оцененных пользователем)

In [84]:
cos_dict_br = sc.broadcast(cos_dict)

def get_similar_films(fid):
    cos = cos_dict_br.value.get(fid, {})
    sorted_cos = sorted(cos.items(), key=itemgetter(1), reverse=True)
    sorted_cos_list = [x[0] for x in sorted_cos[1:31]]
    return sorted_cos_list

In [85]:
# 30 ближайших фильмов
neib = no_rating_films.map(lambda x: {x[0]: get_similar_films(x[0])})
neib_local = neib.collect()

In [88]:
neib_local_dict = {}
for _i in neib_local:
    neib_local_dict[_i.keys()[0]] = _i.values()[0]
    
neib_local_dict[3]

[1230,
 409,
 1677,
 282,
 1231,
 33,
 1552,
 783,
 1659,
 1660,
 1476,
 1443,
 1052,
 1477,
 67,
 1408,
 952,
 559,
 1652,
 860,
 41,
 388,
 421,
 554,
 161,
 1058,
 628,
 1518,
 998,
 552]

####прогноз оценки пользователя по формуле

In [102]:
def prabably_rating_2(film_id, user_id, only_positive=False):
    # первое слагаемое
    b_ui = average_rating_br.value + bi_dict_br.value.get(film_id) + bu_dict_br.value.get(user_id)
    
    # соседи фильма
    film_neib = neib_local_dict.get(film_id, [])
    
    # найдем множество фильмов соседей, по которым у пользователя есть оценка
    neib_ratings = my_user_marks.filter(lambda x: x[0] in film_neib)
    
    # [(559, 3), (161, 1), (952, 5), (33, 3)] - [(фильм, оценка)]
    neib_ratings_local = neib_ratings.collect()
    
    s1 = 0
    s2 = 0
    for _i in neib_ratings_local:
        _cos = cos_dict.get(film_id, {}).get(_i[0])
        if only_positive and _cos <= 0:
            continue
        b_uj = average_rating_br.value + bi_dict_br.value.get(_i[0]) + bu_dict_br.value.get(user_id)
        s1 += _cos * (_i[1] - b_uj)
        s2 += abs(_cos)
    
    _s = 0
    if s2 is not 0:
        _s = round(float(s1) / s2, 4)
    
    return b_ui + _s
    

In [93]:
# считаем
res = []
_counter = 0
for fid in no_rating_films_list:
    res.append((fid, prabably_rating_2(fid, user_id)))
    _counter += 1
    if _counter % 50 == 0:
        print _counter*100 / len(no_rating_films_list)

3
7
10
14
18
21
25
29
32
36
39
43
47
50
54
58
61
65
68
72
76
79
83
87
90
94
97


####Рекомендуйте пользователю 10 фильмов (predicators_top10) с самыми высокими оценками

In [99]:
sorted_prob = sorted(res, key=itemgetter(1), reverse=True)
sorted_prob[:10]

[(1603, 5.2276),
 (251, 5.1921),
 (469, 5.165699999999999),
 (1476, 5.1232),
 (1184, 5.057),
 (1512, 5.0359),
 (922, 5.0296),
 (902, 4.9955),
 (1474, 4.9802),
 (1475, 4.9780999999999995)]

In [100]:
predicators_top10 = [i[0] for i in sorted_prob[:10]]
predicators_top10

[1603, 251, 469, 1476, 1184, 1512, 922, 902, 1474, 1475]

####Пересчитаем оценки, выкинув соседей с отрицательной близостью

In [103]:
res = []
_counter = 0
for fid in no_rating_films_list:
    res.append((fid, prabably_rating_2(fid, user_id, only_positive=True)))
    _counter += 1
    if _counter % 50 == 0:
        print _counter*100 / len(no_rating_films_list)

3
7
10
14
18
21
25
29
32
36
39
43
47
50
54
58
61
65
68
72
76
79
83
87
90
94
97


####

In [104]:
sorted_prob = sorted(res, key=itemgetter(1), reverse=True)
sorted_prob[:10]

[(1603, 5.2276),
 (251, 5.1921),
 (469, 5.165699999999999),
 (1476, 5.1232),
 (1184, 5.057),
 (1512, 5.0359),
 (922, 5.0296),
 (902, 4.9955),
 (1474, 4.9802),
 (1475, 4.9780999999999995)]

In [105]:
predicators_positive_top10 = [i[0] for i in sorted_prob[:10]]
predicators_positive_top10

[1603, 251, 469, 1476, 1184, 1512, 922, 902, 1474, 1475]

In [85]:
# сохраняем
import json

result = {
    "average_film_ratings": average_film_ratings,
    "average_rating": average_rating,
    "average_user_ratings": average_user_ratings,
    "completeness": completeness,
    "pearson_neighbours": pearson_neighbours,
    "pearson_top10": pearson_top10,
    "predicators_positive_top10": predicators_positive_top10,
    "predicators_top10": predicators_top10
}

with open('lab11.json', 'w') as f:
    f.write(json.dumps(result))
    f.close()